In [1]:
import numpy as np
from IPython.display import clear_output
import cv2
import os
from PIL import Image
from sklearn.utils import shuffle
import tkinter as tk
import json

In [2]:
def processX(frame):
    frame = frame.reshape(frame.shape[0],-1)
    frame = frame.reshape(frame.shape[0] * frame.shape[1], 1)
    frame = frame/255
    return frame

In [3]:
data = []
y = np.loadtxt("dataPoints/labels.txt",delimiter = ",")
y = y.reshape(y.shape[0],1)
path = r'dataPoints/All'
counter = 0
scale_percent = 10 # percent of original size
width = int(1280 * scale_percent / 100)
height = int(720 * scale_percent / 100)
dim = (width, height)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        img = cv2.imread(path+'/'+file, cv2.IMREAD_UNCHANGED)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        data.append(resized)
print(len(data))
#Lets look at an example picture
cv2.imshow("Resized image", data[0])
cv2.waitKey(0)
cv2.destroyAllWindows()
print(data[0].shape)

511
(72, 128, 3)


In [4]:
#Convert List of images to one numpy
X_orig = np.array(data)
#Flatten the input
X = X_orig.reshape(X_orig.shape[0],-1).T
X.shape
#Standardize
X = X/255

In [5]:
# shuffle the data
X = X.T
randomize = np.arange(len(X))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]
X = X.T

In [6]:
X.shape

(27648, 511)

In [7]:
def relu(Z):
    return np.maximum(0,Z)

In [8]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

In [9]:
def reluPrime(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0
    return dZ

In [10]:
def sigmoidPrime(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

In [11]:
def processY(Y,num):
    return (Y == num).astype(np.int)

In [12]:
def initParams(dims):
    params = {}
    #initialize the Weights and biases
    np.random.seed(1)
    for l in range(1,len(dims)):
        params["W" + str(l)] = np.random.randn(dims[l],dims[l-1]) * 0.01
        params["b" + str(l)] = np.zeros((dims[l],1))
    #Assert Param sizes
    assert(params['W' + str(l)].shape == (dims[l], dims[l-1]))
    assert(params['b' + str(l)].shape == (dims[l], 1))
    return params

In [13]:
def forwardProp(params,X):
    #One cycle of Forward Prop
    L = int(len(params)/2)
    cache = {}
    A_prev = X
    A = X
    cache["A0"] = X
    for l in range(1,L):
        # Get Vars from the dict
        W = params["W"+str(l)]
        b = params["b"+str(l)]
        # Compute single forward prop step
        Z = np.dot(W,A_prev) + b
        A = relu(Z)
        A_prev = A
        #store in the cache dict
        cache["Z"+str(l)] = Z
        cache["A"+str(l)] = A
        
    #Last Sigmoid Step
    #Get vars
    W = params["W"+str(L)]
    b = params["b"+str(L)]
    #Compute
    Z = np.dot(W,A_prev) + b
    A = sigmoid(Z)
    #store
    cache["Z"+str(L)] = Z
    cache["A"+str(L)] = A
    return cache,A

In [14]:
def cost(Yhat,Y):
    #Compute the cost
    Yhat = Yhat.T
    m = Y.shape[0]
    cost = (-1/m) * np.sum(np.multiply(np.log(Yhat),Y) + np.multiply(np.log(1-Yhat),(1-Y)))
    cost = np.squeeze(cost)
    return cost

In [15]:
def backProp(cache, params, Y, AL):
    # initialization
    grads = {}
    Y = Y.reshape(AL.shape)
    m = (Y.shape[1])
    L = int(len(cache)/2)
    dA = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ZL = cache["Z"+str(L)]
    dZ = sigmoidPrime(dA,ZL)
    dW = (1/m) * np.dot(dZ,cache["A"+str(L-1)].T)
    db = (1/m) * np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(params["W"+str(L)].T,dZ)
    #store
    grads["dW"+str(L)] = dW
    grads["db"+str(L)] = db
    grads["dA"+str(L-1)] = dA_prev
    
    #For the relu function
    for l in range(L-1,0,-1):
        dA = grads["dA"+str(l)]
        Z = cache["Z"+str(l)]
        dZ = reluPrime(dA,Z)
        dW = (1/m) * np.dot(dZ,cache["A"+str(l-1)].T)
        db = (1/m) * np.sum(dZ,axis=1,keepdims=True)
        dA_prev = np.dot(params["W"+str(l)].T,dZ)
        #store em
        grads["dW"+str(l)] = dW
        grads["db"+str(l)] = db
        grads["dA"+str(l-1)] = dA_prev
    
    return grads

In [16]:
def updateParams(params,grads,learning_rate):
    #update params
    L = int(len(params)/2)
    for l in range(L):
        params["W"+str(l+1)] = params["W"+str(l+1)] - learning_rate * grads["dW"+str(l+1)]
        params["b"+str(l+1)] = params["b"+str(l+1)] - learning_rate * grads["db"+str(l+1)]
    return params

In [17]:
def NN_model(dims,X,Y,learning_rate,numIters):
    #initialize Parameters
    params = initParams(dims)
    #now to loop
    for i in range(numIters):
        #forward propagation
        cache,yHat = forwardProp(params,X)
        #back propagation
        grads = backProp(cache,params,Y,yHat)
        #update parameters
        params = updateParams(params,grads,learning_rate)
        #print cost occasionally
        if i%100 == 0:
            costVal = cost(yHat,Y)
            print("Cost after iteration {}: {}".format(i, costVal))
    return params

In [18]:
def predict(X,params):
    confidence=[]
    for i in range(5):
        curr_params = params[i]
        L = int(len(curr_params)/2)
        A_prev = X
        A = X
        for l in range(1,L):
            # Get Vars from the dict
            W = curr_params["W"+str(l)]
            b = curr_params["b"+str(l)]
            # Compute single forward prop step
            Z = np.dot(W,A_prev) + b
            A = relu(Z)
            A_prev = A
        W = curr_params["W"+str(L)]
        b = curr_params["b"+str(L)]
        #Compute
        Z = np.dot(W,A_prev) + b
        A = sigmoid(Z)
        confidence.append(A)
    return confidence

In [19]:
def storeParams(paramsList):
    for i in range(1,6):
        data = paramsList[i-1]
        np.save('parameters/params'+str(i)+'.npy',data)

In [20]:
all_Y = {}
all_Y["Y1"] = processY(y,1)
all_Y["Y2"] = processY(y,2)
all_Y["Y3"] = processY(y,3)
all_Y["Y4"] = processY(y,4)
all_Y["Y5"] = processY(y,5)
allParams = []
for i in range(1,6):
    allParams.append(NN_model([27648,150,70,20,1],X,all_Y["Y"+str(i)],0.03,2500))
storeParams(allParams)

Cost after iteration 0: 0.6931618564875948
Cost after iteration 100: 0.5474529189234908
Cost after iteration 200: 0.5107684219192102
Cost after iteration 300: 0.49989601901502284
Cost after iteration 400: 0.49630390811960196
Cost after iteration 500: 0.4950228655951196
Cost after iteration 600: 0.4945232302405878
Cost after iteration 700: 0.49428158956036405
Cost after iteration 800: 0.49409242824324395
Cost after iteration 900: 0.4938290506402814
Cost after iteration 1000: 0.4932773539039249
Cost after iteration 1100: 0.4916422061318059
Cost after iteration 1200: 0.48246641305821225
Cost after iteration 1300: 0.3725952813710431
Cost after iteration 1400: 0.38011790983552934
Cost after iteration 1500: 0.04132489029080448
Cost after iteration 1600: 0.013441243282025986
Cost after iteration 1700: 0.0055842735571361626
Cost after iteration 1800: 0.003133989639652689
Cost after iteration 1900: 0.0020462569470888997
Cost after iteration 2000: 0.0014627373939903397
Cost after iteration 2100: